Set Environment Variables

In [2]:
!pip install pyspark


In [3]:
import os

# Clear SPARK_HOME if it's set
if 'SPARK_HOME' in os.environ:
    del os.environ['SPARK_HOME']

# Import SparkSession
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("SparkSQL") \
    .getOrCreate()

# Test the SparkSession by creating a DataFrame
df = spark.createDataFrame([(1, "Test"), (2, "Spark")], ["id", "name"])
df.show()


+---+-----+
| id| name|
+---+-----+
|  1| Test|
|  2|Spark|
+---+-----+



In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



# Add the URL to SparkFiles
spark.sparkContext.addFile(url)

# Read the CSV file into a DataFrame
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show the first few rows of the DataFrame to understand its structure
df.show(5)

# Print the schema of the DataFrame to see the data types and column names
df.printSchema()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [6]:
# 2. Create a temporary view of the DataFrame.


df.createOrReplaceTempView("home_sales")

# Example of querying the temporary view using Spark SQL
spark.sql("SELECT * FROM home_sales LIMIT 5").show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [7]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?


query1 = """
SELECT YEAR(date) AS year_sold, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year_sold
ORDER BY year_sold
"""
spark.sql(query1).show()



+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [8]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?


query2 = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""
spark.sql(query2).show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [9]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?



query3 = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""
spark.sql(query3).show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [10]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

import time
start_time = time.time()

query4 = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
spark.sql(query4).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.0703086853027344 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.


# Cache the DataFrame backing the temporary view
spark.sql("CACHE TABLE home_sales")

#check if the table is cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {is_cached}")


Is 'home_sales' cached? True


In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [13]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

import time

# Start the timer
start_time = time.time()

# Execute the query using the cached data
query_cached = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
spark.sql(query_cached).show()

# End the timer and print the runtime
runtime_cached = time.time() - start_time
print(f"--- Cached Query Runtime: {runtime_cached:.2f} seconds ---")



+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- Cached Query Runtime: 0.52 seconds ---


In [14]:
# Clear the cache (for testing purposes to compare)
spark.sql("UNCACHE TABLE home_sales")

# Start the timer for the uncached query
start_time_uncached = time.time()

# Execute the same query without caching
spark.sql(query_cached).show()

# End the timer and print the runtime for the uncached query
runtime_uncached = time.time() - start_time_uncached
print(f"--- Uncached Query Runtime: {runtime_uncached:.2f} seconds ---")


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- Uncached Query Runtime: 0.57 seconds ---


In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data


# Define the path where the Parquet files will be stored
output_path = "/content/home_sales_parquet"

# Write the DataFrame to a Parquet file, partitioned by 'date_built'
df.write.partitionBy("date_built").parquet(output_path)



In [16]:
# 11. Read the formatted parquet data.



# Define the path where your Parquet files are stored

parquet_path = "/content/home_sales_parquet"

# Read the Parquet file into a DataFrame
parquet_df = spark.read.parquet(parquet_path)

# Show the first few rows of the DataFrame to verify it's loaded correctly
parquet_df.show(5)

# print the schema to see how the data types are inferred
parquet_df.printSchema()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [17]:
# 12. Create a temporary table for the parquet data.



# using the parquet_df , the above DataFrame loaded from your Parquet files
parquet_df.createOrReplaceTempView("parquet_home_sales")


# to see the first few rows from the temporary view
spark.sql("SELECT * FROM parquet_home_sales LIMIT 5").show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [18]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

import time

# Start timing
start_time = time.time()

# Execute the SQL query using the temporary view created from the Parquet DataFrame
query_parquet = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM parquet_home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
result = spark.sql(query_parquet)
result.show()

# End timing and print the runtime
runtime_parquet = time.time() - start_time
print(f"--- Parquet Query Runtime: {runtime_parquet:.2f} seconds ---")


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- Parquet Query Runtime: 0.67 seconds ---


In [20]:
# 14. Uncache the home_sales temporary table.


# Uncache the home_sales temporary table
spark.sql("UNCACHE TABLE home_sales")

# Check if the table is cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {is_cached}")


Is 'home_sales' cached? False


In [21]:
# 15. Check if the home_sales is no longer cached


# Check if the 'home_sales' table is cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {is_cached}")



Is 'home_sales' cached? False
